In [2]:
import json
import random
with open('training.json') as json_file:  
    train_json = json.load(json_file)
with open('development.json') as json_file:  
    dev_json = json.load(json_file)
with open('testing.json') as json_file:  
    test_json = json.load(json_file)
with open('sample.json') as json_file:  
    sample_json = json.load(json_file)
    
def get_dic_from_two_lists(keys, values):
    return { keys[i] : values[i] for i in range(len(keys)) }

In [15]:
data = []
for article in dev_json['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        context = context.replace("''", '" ')
        context = context.replace("``", '" ')
        context_map = {}
        qas = []
        for qa in paragraph['qas']:           
            question = qa['question']
            answer = qa['answers'][0]
            ans_text = answer['text']
            ID = qa['id']
            qas.append((ID,question,ans_text))
        data.append((context,qas))

In [17]:
import spacy
from nltk import sent_tokenize
import en_core_web_sm
nlp = en_core_web_sm.load()

In [18]:
contexts = [a for (a,b) in data]
dev_data = []
for i,context in enumerate(contexts):
    candidates  = list(nlp(context).noun_chunks)
    sentences = sent_tokenize(context)
    for q_id,question,answer in data[i][1]:
        candidate_tuples = [i,question,q_id]
        contain_answer = False
        for candidate_answer in candidates:  
            candidate_sentence=''
            for sentence in sentences:
                if str(candidate_answer) in sentence:
                    candidate_sentence = sentence
            if answer in str(candidate_answer):
                candidate_tuples.extend([candidate_sentence,candidate_answer,1])
                contain_answer = True
            else:
                candidate_tuples.extend([candidate_sentence,candidate_answer,0])
        if not contain_answer:
            for sentence in sentences:
                if answer in sentence:
                    candidate_sentence = sentence
            candidate_tuples.extend([candidate_sentence,answer,1])
        dev_data.append(candidate_tuples)

In [268]:
import csv
with open('train_data.csv','w') as out:
    csv_out=csv.writer(out)
    for row in dev_data:
        csv_out.writerow(row)

In [269]:
with open('train_contexts.txt','w') as out:
    out.write("\n".join(contexts))